# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia DataFrames CSV"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2022-02-17 12:24:09 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package DataFrames...
Installing Julia package CSV...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [ ]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.00GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake-avx512)
Environment:
  JULIA_NUM_THREADS = 4


# Step 1. Import the necessary libraries

In [ ]:
using DataFrames
using Statistics


# Step 2. Create the 3 DataFrames based on the following raw data

In [ ]:
raw_data_1 = Dict(:subject_id => ["1", "2", "3", "4", "5"],
    :first_name => ["Alex", "Amy", "Allen", "Alice", "Ayoung"],
    :last_name => ["Anderson", "Ackerman", "Ali", "Aoni", "Atiches"])

raw_data_2 = Dict(:subject_id => ["4", "5", "6", "7", "8"],
    :first_name => ["Billy", "Brian", "Bran", "Bryce", "Betty"],
    :last_name => ["Bonder", "Black", "Balwner", "Brice", "Btisan"])

raw_data_3 = Dict(
    :subject_id => ["1", "2", "3", "4", "5", "7", "8", "9", "10", "11"],
    :test_id => [51, 15, 15, 61, 16, 14, 15, 1, 61, 16])

Dict{Symbol, Vector{T} where T} with 2 entries:
  :subject_id => ["1", "2", "3", "4", "5", "7", "8", "9", "10", "11"]
  :test_id    => [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]

# Step 3. Assign each to a variable called data1, data2, data3

In [ ]:
data1 = DataFrame(; raw_data_1...)
data2 = DataFrame(; raw_data_2...)
data3 = DataFrame(; raw_data_3...)
show(data1)

5×3 DataFrame
 Row │ subject_id  last_name  first_name 
     │ String      String     String     
─────┼───────────────────────────────────
   1 │ 1           Anderson   Alex
   2 │ 2           Ackerman   Amy
   3 │ 3           Ali        Allen
   4 │ 4           Aoni       Alice
   5 │ 5           Atiches    Ayoung

# Step 4. Join the two dataframes along rows and assign all_data

In [ ]:
all_data = vcat(data1, data2)

,subject_id,last_name,first_name
,String,String,String
1,1,Anderson,Alex
2,2,Ackerman,Amy
3,3,Ali,Allen
4,4,Aoni,Alice
5,5,Atiches,Ayoung
6,4,Bonder,Billy
7,5,Black,Brian
8,6,Balwner,Bran
9,7,Brice,Bryce


# Step 5. Join the two dataframes along columns and assing to all_data_col

In [ ]:
all_data_col = hcat(data1, data2, makeunique = true)

,subject_id,last_name,first_name,subject_id_1,last_name_1,first_name_1
,String,String,String,String,String,String
1,1,Anderson,Alex,4,Bonder,Billy
2,2,Ackerman,Amy,5,Black,Brian
3,3,Ali,Allen,6,Balwner,Bran
4,4,Aoni,Alice,7,Brice,Bryce
5,5,Atiches,Ayoung,8,Btisan,Betty


# Step 6. Print data3

In [ ]:
show(data3)

10×2 DataFrame
 Row │ test_id  subject_i 
     │ Int64    String    
─────┼────────────────────
   1 │      51  1
   2 │      15  2
   3 │      15  3
   4 │      61  4
   5 │      16  5
   6 │      14  7
   7 │      15  8
   8 │       1  9
   9 │      61  10
  10 │      16  11

# Step 7. Merge all_data and data3 along the subject_id value


In [ ]:
innerjoin(all_data, data3, on = :subject_id)

,subject_id,last_name,first_name,test_id
,String,String,String,Int64
1,1,Anderson,Alex,51
2,2,Ackerman,Amy,15
3,3,Ali,Allen,15
4,4,Aoni,Alice,61
5,4,Bonder,Billy,61
6,5,Atiches,Ayoung,16
7,5,Black,Brian,16
8,7,Brice,Bryce,14
9,8,Btisan,Betty,15


# Step 8. Merge only the data that has the same 'subject_id' on both data1 and data2

In [ ]:
innerjoin(data1, data2, on = :subject_id, makeunique = true)

,subject_id,last_name,first_name,last_name_1,first_name_1
,String,String,String,String,String
1,4,Aoni,Alice,Bonder,Billy
2,5,Atiches,Ayoung,Black,Brian


# Step 9. Merge all values in data1 and data2, with matching records from both sides where available.

In [ ]:
outerjoin(data1, data2, on = :subject_id, makeunique = true)

,subject_id,last_name,first_name,last_name_1,first_name_1
,String,String?,String?,String?,String?
1,4,Aoni,Alice,Bonder,Billy
2,5,Atiches,Ayoung,Black,Brian
3,1,Anderson,Alex,missing,missing
4,2,Ackerman,Amy,missing,missing
5,3,Ali,Allen,missing,missing
6,6,missing,missing,Balwner,Bran
7,7,missing,missing,Brice,Bryce
8,8,missing,missing,Btisan,Betty
